In [1]:
import io
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from glob import glob
from PIL import Image
from tqdm import tqdm
from pycocotools.coco import COCO
from tensorflow import keras
from keras.applications import ResNet101
from keras.layers import Conv2D, Conv1D, BatchNormalization, Activation, UpSampling2D, AveragePooling2D, Concatenate, Input, Softmax, ReLU, Embedding, TextVectorization
from keras.models import Model
from tensorflow import linalg
from keras.optimizers import SGD
from tqdm.notebook import tqdm
import math
import shutil

C:\Users\SamuelMcmurray\.conda\envs\tf\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [3]:
color_map = []
for i in range(173):
    i = (i * 255 / 255)
    r = round(math.sin(0.024 * i + 0) * 127 + 128)
    g = round(math.sin(0.024 * i + 2) * 127 + 128)
    b = round(math.sin(0.024 * i + 4) * 127 + 128)
    color_map.append((r,g,b))

for i in color_map:
    print(i)

print(len(color_map))

(128, 243, 32)
(131, 242, 30)
(134, 241, 28)
(137, 239, 26)
(140, 238, 24)
(143, 236, 23)
(146, 235, 21)
(149, 233, 19)
(152, 231, 18)
(155, 229, 16)
(158, 228, 15)
(161, 226, 14)
(164, 224, 12)
(167, 222, 11)
(170, 220, 10)
(173, 217, 9)
(176, 215, 8)
(178, 213, 7)
(181, 211, 6)
(184, 208, 5)
(187, 206, 4)
(189, 204, 4)
(192, 201, 3)
(195, 199, 3)
(197, 196, 2)
(200, 193, 2)
(202, 191, 1)
(205, 188, 1)
(207, 185, 1)
(209, 183, 1)
(212, 180, 1)
(214, 177, 1)
(216, 174, 1)
(218, 171, 1)
(221, 169, 2)
(223, 166, 2)
(225, 163, 2)
(227, 160, 3)
(228, 157, 4)
(230, 154, 4)
(232, 151, 5)
(234, 148, 6)
(235, 145, 7)
(237, 142, 7)
(239, 139, 8)
(240, 136, 9)
(241, 133, 11)
(243, 130, 12)
(244, 127, 13)
(245, 124, 14)
(246, 121, 16)
(247, 118, 17)
(248, 115, 19)
(249, 111, 20)
(250, 108, 22)
(251, 105, 24)
(252, 102, 26)
(252, 99, 27)
(253, 97, 29)
(253, 94, 31)
(254, 91, 33)
(254, 88, 35)
(255, 85, 37)
(255, 82, 39)
(255, 79, 42)
(255, 76, 44)
(255, 74, 46)
(255, 71, 49)
(255, 68, 51)
(255, 66

In [4]:
things_dict = {
    'things': 'things',
    'indoor-super-things': 'things',
    'outdoor-super-things': 'things',
    'person-things': 'outdoor-super-things',
    'vehicle-things': 'outdoor-super-things',
    'outdoor-things': 'outdoor-super-things',
    'animal-things': 'outdoor-super-things',
    'accessory-things': 'outdoor-super-things',
    'sports-things': 'outdoor-super-things',
    'kitchen-things': 'indoor-super-things',
    'food-things': 'indoor-super-things',
    'furniture-things': 'indoor-super-things',
    'electronic-things': 'indoor-super-things',
    'appliance-things': 'indoor-super-things',
    'indoor-things': 'indoor-super-things',
    'person': 'person-things',
    'bicycle': 'vehicle-things',
    'car': 'vehicle-things',
    'motorcycle': 'vehicle-things',
    'airplane': 'vehicle-things',
    'bus': 'vehicle-things',
    'train': 'vehicle-things',
    'truck': 'vehicle-things',
    'boat': 'vehicle-things',
    'traffic light': 'outdoor-things',
    'fire hydrant': 'outdoor-things',
    'street sign': 'outdoor-things',
    'stop sign': 'outdoor-things',
    'parking meter': 'outdoor-things',
    'bench': 'outdoor-things',
    'bird': 'animal-things',
    'cat': 'animal-things',
    'dog': 'animal-things',
    'horse': 'animal-things',
    'sheep': 'animal-things',
    'cow': 'animal-things',
    'elephant': 'animal-things',
    'bear': 'animal-things',
    'zebra': 'animal-things',
    'giraffe': 'animal-things',
    'hat': 'accessory-things',
    'backpack': 'accessory-things',
    'umbrella': 'accessory-things',
    'shoe': 'accessory-things',
    'eye glasses': 'accessory-things',
    'handbag': 'accessory-things',
    'tie': 'accessory-things',
    'suitcase': 'accessory-things',
    'frisbee': 'sports-things',
    'skis': 'sports-things',
    'snowboard': 'sports-things',
    'sports ball': 'sports-things',
    'kite': 'sports-things',
    'baseball bat': 'sports-things',
    'baseball glove': 'sports-things',
    'skateboard': 'sports-things',
    'surfboard': 'sports-things',
    'tennis racket': 'sports-things',
    'bottle': 'kitchen-things',
    'plate': 'kitchen-things',
    'wine glass': 'kitchen-things',
    'cup': 'kitchen-things',
    'fork': 'kitchen-things',
    'knife': 'kitchen-things',
    'spoon': 'kitchen-things',
    'bowl': 'kitchen-things',
    'banana': 'food-things',
    'apple': 'food-things',
    'sandwich': 'food-things',
    'orange': 'food-things',
    'broccoli': 'food-things',
    'carrot': 'food-things',
    'hot dog': 'food-things',
    'pizza': 'food-things',
    'donut': 'food-things',
    'cake': 'food-things',
    'chair': 'furniture-things',
    'couch': 'furniture-things',
    'potted plant': 'furniture-things',
    'bed': 'furniture-things',
    'mirror': 'furniture-things',
    'dining table': 'furniture-things',
    'window': 'furniture-things',
    'desk': 'furniture-things',
    'toilet': 'furniture-things',
    'door': 'furniture-things',
    'tv': 'electronic-things',
    'laptop': 'electronic-things',
    'mouse': 'electronic-things',
    'remote': 'electronic-things',
    'keyboard': 'electronic-things',
    'cell phone': 'electronic-things',
    'microwave': 'appliance-things',
    'oven': 'appliance-things',
    'toaster': 'appliance-things',
    'sink': 'appliance-things',
    'refrigerator': 'appliance-things',
    'blender': 'appliance-things',
    'book': 'indoor-things',
    'clock': 'indoor-things',
    'vase': 'indoor-things',
    'scissors': 'indoor-things',
    'teddy bear': 'indoor-things',
    'hair drier': 'indoor-things',
    'toothbrush': 'indoor-things',
    'hair brush': 'indoor-things'

}

In [5]:
stuff_dict = {
    'stuff': 'stuff',
    'indoor-super-stuff': 'stuff',
    'outdoor-super-stuff': 'stuff',
    'rawmaterial-stuff': 'indoor-super-stuff',
    'wall-stuff': 'indoor-super-stuff',
    'ceiling-stuff': 'indoor-super-stuff',
    'floor-stuff': 'indoor-super-stuff',
    'window-stuff': 'indoor-super-stuff',
    'furniture-stuff': 'indoor-super-stuff',
    'textile-stuff': 'indoor-super-stuff',
    'food-stuff': 'indoor-super-stuff',
    'building-stuff': 'outdoor-super-stuff',
    'structural-stuff': 'outdoor-super-stuff',
    'plant-stuff': 'outdoor-super-stuff',
    'sky-stuff': 'outdoor-super-stuff',
    'solid-stuff': 'outdoor-super-stuff',
    'ground-stuff': 'outdoor-super-stuff',
    'water-stuff': 'outdoor-super-stuff',
    'cardboard': 'rawmaterial-stuff',
    'paper': 'rawmaterial-stuff',
    'plastic': 'rawmaterial-stuff',
    'metal': 'rawmaterial-stuff',
    'wall-tile': 'wall-stuff',
    'wall-panel': 'wall-stuff',
    'wall-wood': 'wall-stuff',
    'wall-brick': 'wall-stuff',
    'wall-stone': 'wall-stuff',
    'wall-concrete': 'wall-stuff',
    'wall-other': 'wall-stuff',
    'ceiling-tile': 'ceiling-stuff',
    'ceiling-other': 'ceiling-stuff',
    'carpet': 'floor-stuff',
    'floor-tile': 'floor-stuff',
    'floor-wood': 'floor-stuff',
    'floor-marble': 'floor-stuff',
    'floor-stone': 'floor-stuff',
    'floor-other': 'floor-stuff',
    'window-blind': 'window-stuff',
    'window-other': 'window-stuff',
    'door-stuff': 'furniture-stuff',
    'desk-stuff': 'furniture-stuff',
    'table': 'furniture-stuff',
    'shelf': 'furniture-stuff',
    'cabinet': 'furniture-stuff',
    'cupboard': 'furniture-stuff',
    'mirror-stuff': 'furniture-stuff',
    'counter': 'furniture-stuff',
    'light': 'furniture-stuff',
    'stairs': 'furniture-stuff',
    'furniture-other': 'furniture-stuff',
    'rug': 'textile-stuff',
    'mat': 'textile-stuff',
    'towel': 'textile-stuff',
    'napkin': 'textile-stuff',
    'clothes': 'textile-stuff',
    'cloth': 'textile-stuff',
    'curtain': 'textile-stuff',
    'blanket': 'textile-stuff',
    'pillow': 'textile-stuff',
    'banner': 'textile-stuff',
    'textile-other': 'textile-stuff',
    'fruit': 'food-stuff',
    'salad': 'food-stuff',
    'vegetable': 'food-stuff',
    'food-other': 'food-stuff',
    'house': 'building-stuff',
    'skyscraper': 'building-stuff',
    'bridge': 'building-stuff',
    'tent': 'building-stuff',
    'roof': 'building-stuff',
    'building-other': 'building-stuff',
    'fence': 'structural-stuff',
    'cage': 'structural-stuff',
    'net': 'structural-stuff',
    'railing': 'structural-stuff',
    'structural-other': 'structural-stuff',
    'grass': 'plant-stuff',
    'tree': 'plant-stuff',
    'bush': 'plant-stuff',
    'leaves': 'plant-stuff',
    'flower': 'plant-stuff',
    'branch': 'plant-stuff',
    'moss': 'plant-stuff',
    'straw': 'plant-stuff',
    'plant-other': 'plant-stuff',
    'clouds': 'sky-stuff',
    'sky-other': 'sky-stuff',
    'wood': 'solid-stuff',
    'rock': 'solid-stuff',
    'stone': 'solid-stuff',
    'mountain': 'solid-stuff',
    'hill': 'solid-stuff',
    'solid-other': 'solid-stuff',
    'sand': 'ground-stuff',
    'snow': 'ground-stuff',
    'dirt': 'ground-stuff',
    'mud': 'ground-stuff',
    'gravel': 'ground-stuff',
    'road': 'ground-stuff',
    'pavement': 'ground-stuff',
    'railroad': 'ground-stuff',
    'platform': 'ground-stuff',
    'playingfield': 'ground-stuff',
    'ground-other': 'ground-stuff',
    'fog': 'water-stuff',
    'river': 'water-stuff',
    'sea': 'water-stuff',
    'waterdrops': 'water-stuff',
    'water-other': 'water-stuff',
}

In [6]:
all_dict = {
    'stuff': 'stuff',
    'indoor': 'stuff',
    'outdoor': 'stuff',
    'rawmaterial': 'indoor',
    'wall': 'indoor',
    'ceiling': 'indoor',
    'floor': 'indoor',
    'window': 'indoor',
    'furniture': 'indoor',
    'textile': 'indoor',
    'food': 'indoor',
    'building': 'outdoor',
    'structural': 'outdoor',
    'plant': 'outdoor',
    'sky': 'outdoor',
    'solid': 'outdoor',
    'ground': 'outdoor',
    'water': 'outdoor',
    'cardboard': 'rawmaterial',
    'paper': 'rawmaterial',
    'plastic': 'rawmaterial',
    'metal': 'rawmaterial',
    'wall-tile': 'wall',
    'wall-panel': 'wall',
    'wall-wood': 'wall',
    'wall-brick': 'wall',
    'wall-stone': 'wall',
    'wall-concrete': 'wall',
    'wall-other': 'wall',
    'ceiling-tile': 'ceiling',
    'ceiling-other': 'ceiling',
    'carpet': 'floor',
    'floor-tile': 'floor',
    'floor-wood': 'floor',
    'floor-marble': 'floor',
    'floor-stone': 'floor',
    'floor-other': 'floor',
    'window-blind': 'window',
    'window-other': 'window',
    'door': 'furniture',
    'desk': 'furniture',
    'table': 'furniture',
    'shelf': 'furniture',
    'cabinet': 'furniture',
    'cupboard': 'furniture',
    'mirror': 'furniture',
    'counter': 'furniture',
    'light': 'furniture',
    'stairs': 'furniture',
    'furniture-other': 'furniture',
    'rug': 'textile',
    'mat': 'textile',
    'towel': 'textile',
    'napkin': 'textile',
    'clothes': 'textile',
    'cloth': 'textile',
    'curtain': 'textile',
    'blanket': 'textile',
    'pillow': 'textile',
    'banner': 'textile',
    'textile-other': 'textile',
    'fruit': 'food',
    'salad': 'food',
    'vegetable': 'food',
    'food-other': 'food',
    'house': 'building',
    'skyscraper': 'building',
    'bridge': 'building',
    'tent': 'building',
    'roof': 'building',
    'building-other': 'building',
    'fence': 'structural',
    'cage': 'structural',
    'net': 'structural',
    'railing': 'structural',
    'structural-other': 'structural',
    'grass': 'plant',
    'tree': 'plant',
    'bush': 'plant',
    'leaves': 'plant',
    'flower': 'plant',
    'branch': 'plant',
    'moss': 'plant',
    'straw': 'plant',
    'plant-other': 'plant',
    'clouds': 'sky',
    'sky-other': 'sky',
    'wood': 'solid',
    'rock': 'solid',
    'stone': 'solid',
    'mountain': 'solid',
    'hill': 'solid',
    'solid-other': 'solid',
    'sand': 'ground',
    'snow': 'ground',
    'dirt': 'ground',
    'mud': 'ground',
    'gravel': 'ground',
    'road': 'ground',
    'pavement': 'ground',
    'railroad': 'ground',
    'platform': 'ground',
    'playingfield': 'ground',
    'ground-other': 'ground',
    'fog': 'water',
    'river': 'water',
    'sea': 'water',
    'waterdrops': 'water',
    'water-other': 'water',
    'things': 'things',
    'indoor': 'things',
    'outdoor': 'things',
    'person': 'outdoor',
    'vehicle': 'outdoor',
    'outdoor': 'outdoor',
    'animal': 'outdoor',
    'accessory': 'outdoor',
    'sports': 'outdoor',
    'kitchen': 'indoor',
    'food': 'indoor',
    'furniture': 'indoor',
    'electronic': 'indoor',
    'appliance': 'indoor',
    'indoor': 'indoor',
    'person': 'person',
    'bicycle': 'vehicle',
    'car': 'vehicle',
    'motorcycle': 'vehicle',
    'airplane': 'vehicle',
    'bus': 'vehicle',
    'train': 'vehicle',
    'truck': 'vehicle',
    'boat': 'vehicle',
    'traffic light': 'outdoor',
    'fire hydrant': 'outdoor',
    'street sign': 'outdoor',
    'stop sign': 'outdoor',
    'parking meter': 'outdoor',
    'bench': 'outdoor',
    'bird': 'animal',
    'cat': 'animal',
    'dog': 'animal',
    'horse': 'animal',
    'sheep': 'animal',
    'cow': 'animal',
    'elephant': 'animal',
    'bear': 'animal',
    'zebra': 'animal',
    'giraffe': 'animal',
    'hat': 'accessory',
    'backpack': 'accessory',
    'umbrella': 'accessory',
    'shoe': 'accessory',
    'eye glasses': 'accessory',
    'handbag': 'accessory',
    'tie': 'accessory',
    'suitcase': 'accessory',
    'frisbee': 'sports',
    'skis': 'sports',
    'snowboard': 'sports',
    'sports ball': 'sports',
    'kite': 'sports',
    'baseball bat': 'sports',
    'baseball glove': 'sports',
    'skateboard': 'sports',
    'surfboard': 'sports',
    'tennis racket': 'sports',
    'bottle': 'kitchen',
    'plate': 'kitchen',
    'wine glass': 'kitchen',
    'cup': 'kitchen',
    'fork': 'kitchen',
    'knife': 'kitchen',
    'spoon': 'kitchen',
    'bowl': 'kitchen',
    'banana': 'food',
    'apple': 'food',
    'sandwich': 'food',
    'orange': 'food',
    'broccoli': 'food',
    'carrot': 'food',
    'hot dog': 'food',
    'pizza': 'food',
    'donut': 'food',
    'cake': 'food',
    'chair': 'furniture',
    'couch': 'furniture',
    'potted plant': 'furniture',
    'bed': 'furniture',
    'mirror': 'furniture',
    'dining table': 'furniture',
    'window': 'furniture',
    'desk': 'furniture',
    'toilet': 'furniture',
    'door': 'furniture',
    'tv': 'electronic',
    'laptop': 'electronic',
    'mouse': 'electronic',
    'remote': 'electronic',
    'keyboard': 'electronic',
    'cell phone': 'electronic',
    'microwave': 'appliance',
    'oven': 'appliance',
    'toaster': 'appliance',
    'sink': 'appliance',
    'refrigerator': 'appliance',
    'blender': 'appliance',
    'book': 'indoor',
    'clock': 'indoor',
    'vase': 'indoor',
    'scissors': 'indoor',
    'teddy bear': 'indoor',
    'hair drier': 'indoor',
    'toothbrush': 'indoor',
    'hair brush': 'indoor',
}
print(len(all_dict))

204


In [7]:
# dataDir='..'
# dataType='train2017'
# annFile='annotations_trainval2017/annotations/instances_{}.json'.format(dataType)

In [8]:
# coco = COCO(annotation_file=annFile)

In [9]:
# catIDs = coco.getCatIds()
# cats = coco.loadCats(catIDs)
# print(cats)

In [10]:
# def getClassName(classID, cats):
#     for i in range(len(cats)):
#         if cats[i]['id'] == classID:
#             return cats[i]['name']
#     return "None"

In [11]:
# filter_classes = {0:'unlabeled'}
# filter_classes_map = {'unlabeled': 0}
# for i in range(len(cats)):
#     filter_classes[cats[i]['id']] = cats[i]['name']
#     filter_classes_map[cats[i]['name']] = cats[i]['id']
# print(filter_classes)
# print(len(filter_classes))
# print(filter_classes_map)

In [12]:
# dataDir='stuff_trainval2017'
# dataType='val2017'
# annFile='{}/stuff_{}.json'.format(dataDir, dataType)
#
# # Initialize COCO ground truth API
# cocoGt = COCO(annFile)

In [13]:
# cat_stuffIDs= cocoGt.getCatIds()
# cats_stuff = cocoGt.loadCats(cat_stuffIDs)
# print(cats_stuff)

In [14]:
# for i in range(len(cats_stuff)):
#     filter_classes[cats_stuff[i]['id']] = cats_stuff[i]['name']
#     filter_classes_map[cats_stuff[i]['name']] = cats_stuff[i]['id']
# print(filter_classes)
# print(len(filter_classes))
# print(filter_classes_map)

In [15]:
# print(coco.info())
# print(len(coco.dataset))
# print(cocoGt.info())
# print(len(cocoGt.dataset))
# print("coco: ", len(coco.imgs), " COCOGT: ", len(cocoGt.imgs))

In [16]:
# all_image_ids = []
#
# if filter_classes != None:
#     for i, cl in enumerate(filter_classes_map):
#         if i < 81:
#             print("things: ", cl, i)
#             catIds = coco.getCatIds(catNms=cl)
#             imgIds = coco.getImgIds(catIds=catIds)
#             # images.append(coco.loadImgs(imgIds)[0])
#         else:
#             print("stuff: ", cl, i)
#             all_image_ids.append(imgIds)
#             catIds = cocoGt.getCatIds(catNms=cl)
#             imgIds = cocoGt.getImgIds(catIds=catIds)
#             # images.append(cocoGt.loadImgs(imgIds))
#             all_image_ids.append(imgIds)
# else:
#     imgIds = coco.getImgIds()
#     images = coco.loadImgs(imgIds)

In [17]:
# unique_images = []
# for img_ids in all_image_ids:
#     print("Image ID lengths",len(img_ids))
#     for img_id in img_ids:
#         if img_id not in unique_images:
#             unique_images.append(img_id)
#             print("Unique Images length: ",len(unique_images))
# dataset_size = len(unique_images)
# print("Number of Images in the dataset: ", dataset_size)

In [18]:
# color_mapping = {0: (0,0,0)}
# for i, cl in enumerate(filter_classes_map):
#     id = filter_classes_map[cl]
#     if i == 0:
#         continue
#     color_mapping[id] = color_map[i]
#
# print(color_mapping)

In [19]:
# with open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\val_ids.txt', 'w') as f:
#     for line in unique_images:
#         f.write(str(line))
#         f.write('\n')
#     f.close()

In [20]:
# with open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\color_map.txt', 'w') as f:
#     for i in filter_classes_map:
#         id = filter_classes_map[i]
#         for j, color in enumerate(color_map):
#             if j == id:
#                 line = str(str(i) + " : " + str(id) + "," + str(color[0]) + "," + str(color[1]) + "," + str(color[2]))
#                 f.write(line)
#                 f.write('\n')
#
# f.close()
# print(color_mapping)

In [21]:
# unique_images = []
# file = open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\train_ids.txt', 'r')
# data = file.readlines()
# for i in data:
#     print(i)
#     unique_images.append(int(i))
#
# print(unique_images)

In [22]:
# color_mapping = {0:(0,0,0)}
# file = open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\color_map.txt', 'r')
# data = file.readlines()
#
# for line in data:
#     id_rgb = line.split(":")[1]
#     digits = id_rgb.split(",")
#     id = int(digits[0])
#     r = int(digits[1])
#     g = int(digits[2])
#     b = int(digits[3])
#     color_mapping[id] = (r, g, b)
#
# print(color_mapping)
# print(len(color_mapping))

In [23]:
filter_classes = {0: 'unlabeled', 1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven', 80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock', 86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush', 92: 'banner', 93: 'blanket', 94: 'branch', 95: 'bridge', 96: 'building-other', 97: 'bush', 98: 'cabinet', 99: 'cage', 100: 'cardboard', 101: 'carpet', 102: 'ceiling-other', 103: 'ceiling-tile', 104: 'cloth', 105: 'clothes', 106: 'clouds', 107: 'counter', 108: 'cupboard', 109: 'curtain', 110: 'desk-stuff', 111: 'dirt', 112: 'door-stuff', 113: 'fence', 114: 'floor-marble', 115: 'floor-other', 116: 'floor-stone', 117: 'floor-tile', 118: 'floor-wood', 119: 'flower', 120: 'fog', 121: 'food-other', 122: 'fruit', 123: 'furniture-other', 124: 'grass', 125: 'gravel', 126: 'ground-other', 127: 'hill', 128: 'house', 129: 'leaves', 130: 'light', 131: 'mat', 132: 'metal', 133: 'mirror-stuff', 134: 'moss', 135: 'mountain', 136: 'mud', 137: 'napkin', 138: 'net', 139: 'paper', 140: 'pavement', 141: 'pillow', 142: 'plant-other', 143: 'plastic', 144: 'platform', 145: 'playingfield', 146: 'railing', 147: 'railroad', 148: 'river', 149: 'road', 150: 'rock', 151: 'roof', 152: 'rug', 153: 'salad', 154: 'sand', 155: 'sea', 156: 'shelf', 157: 'sky-other', 158: 'skyscraper', 159: 'snow', 160: 'solid-other', 161: 'stairs', 162: 'stone', 163: 'straw', 164: 'structural-other', 165: 'table', 166: 'tent', 167: 'textile-other', 168: 'towel', 169: 'tree', 170: 'vegetable', 171: 'wall-brick', 172: 'wall-concrete', 173: 'wall-other', 174: 'wall-panel', 175: 'wall-stone', 176: 'wall-tile', 177: 'wall-wood', 178: 'water-other', 179: 'waterdrops', 180: 'window-blind', 181: 'window-other', 182: 'wood', 183: 'other'}

filter_classes_map = {'unlabeled': 0, 'person': 1, 'bicycle': 2, 'car': 3, 'motorcycle': 4, 'airplane': 5, 'bus': 6, 'train': 7, 'truck': 8, 'boat': 9, 'traffic light': 10, 'fire hydrant': 11, 'stop sign': 13, 'parking meter': 14, 'bench': 15, 'bird': 16, 'cat': 17, 'dog': 18, 'horse': 19, 'sheep': 20, 'cow': 21, 'elephant': 22, 'bear': 23, 'zebra': 24, 'giraffe': 25, 'backpack': 27, 'umbrella': 28, 'handbag': 31, 'tie': 32, 'suitcase': 33, 'frisbee': 34, 'skis': 35, 'snowboard': 36, 'sports ball': 37, 'kite': 38, 'baseball bat': 39, 'baseball glove': 40, 'skateboard': 41, 'surfboard': 42, 'tennis racket': 43, 'bottle': 44, 'wine glass': 46, 'cup': 47, 'fork': 48, 'knife': 49, 'spoon': 50, 'bowl': 51, 'banana': 52, 'apple': 53, 'sandwich': 54, 'orange': 55, 'broccoli': 56, 'carrot': 57, 'hot dog': 58, 'pizza': 59, 'donut': 60, 'cake': 61, 'chair': 62, 'couch': 63, 'potted plant': 64, 'bed': 65, 'dining table': 67, 'toilet': 70, 'tv': 72, 'laptop': 73, 'mouse': 74, 'remote': 75, 'keyboard': 76, 'cell phone': 77, 'microwave': 78, 'oven': 79, 'toaster': 80, 'sink': 81, 'refrigerator': 82, 'book': 84, 'clock': 85, 'vase': 86, 'scissors': 87, 'teddy bear': 88, 'hair drier': 89, 'toothbrush': 90, 'banner': 92, 'blanket': 93, 'branch': 94, 'bridge': 95, 'building-other': 96, 'bush': 97, 'cabinet': 98, 'cage': 99, 'cardboard': 100, 'carpet': 101, 'ceiling-other': 102, 'ceiling-tile': 103, 'cloth': 104, 'clothes': 105, 'clouds': 106, 'counter': 107, 'cupboard': 108, 'curtain': 109, 'desk-stuff': 110, 'dirt': 111, 'door-stuff': 112, 'fence': 113, 'floor-marble': 114, 'floor-other': 115, 'floor-stone': 116, 'floor-tile': 117, 'floor-wood': 118, 'flower': 119, 'fog': 120, 'food-other': 121, 'fruit': 122, 'furniture-other': 123, 'grass': 124, 'gravel': 125, 'ground-other': 126, 'hill': 127, 'house': 128, 'leaves': 129, 'light': 130, 'mat': 131, 'metal': 132, 'mirror-stuff': 133, 'moss': 134, 'mountain': 135, 'mud': 136, 'napkin': 137, 'net': 138, 'paper': 139, 'pavement': 140, 'pillow': 141, 'plant-other': 142, 'plastic': 143, 'platform': 144, 'playingfield': 145, 'railing': 146, 'railroad': 147, 'river': 148, 'road': 149, 'rock': 150, 'roof': 151, 'rug': 152, 'salad': 153, 'sand': 154, 'sea': 155, 'shelf': 156, 'sky-other': 157, 'skyscraper': 158, 'snow': 159, 'solid-other': 160, 'stairs': 161, 'stone': 162, 'straw': 163, 'structural-other': 164, 'table': 165, 'tent': 166, 'textile-other': 167, 'towel': 168, 'tree': 169, 'vegetable': 170, 'wall-brick': 171, 'wall-concrete': 172, 'wall-other': 173, 'wall-panel': 174, 'wall-stone': 175, 'wall-tile': 176, 'wall-wood': 177, 'water-other': 178, 'waterdrops': 179, 'window-blind': 180, 'window-other': 181, 'wood': 182, 'other': 183}

COLOR_MAP = [[0, 0, 0], [131, 242, 30], [134, 241, 28], [137, 239, 26], [140, 238, 24], [143, 236, 23], [146, 235, 21], [149, 233, 19], [152, 231, 18], [155, 229, 16], [158, 228, 15], [161, 226, 14], [164, 224, 12], [167, 222, 11], [170, 220, 10], [173, 217, 9], [176, 215, 8], [178, 213, 7], [181, 211, 6], [184, 208, 5], [187, 206, 4], [189, 204, 4], [192, 201, 3], [195, 199, 3], [197, 196, 2], [200, 193, 2], [202, 191, 1], [205, 188, 1], [207, 185, 1], [209, 183, 1], [212, 180, 1], [214, 177, 1], [216, 174, 1], [218, 171, 1], [221, 169, 2], [223, 166, 2], [225, 163, 2], [227, 160, 3], [228, 157, 4], [230, 154, 4], [232, 151, 5], [234, 148, 6], [235, 145, 7], [237, 142, 7], [239, 139, 8], [240, 136, 9], [241, 133, 11], [243, 130, 12], [244, 127, 13], [245, 124, 14], [246, 121, 16], [247, 118, 17], [248, 115, 19], [249, 111, 20], [250, 108, 22], [251, 105, 24], [252, 102, 26], [252, 99, 27], [253, 97, 29], [253, 94, 31], [254, 91, 33], [254, 88, 35], [255, 85, 37], [255, 82, 39], [255, 79, 42], [255, 76, 44], [255, 74, 46], [255, 71, 49], [255, 68, 51], [255, 66, 53], [254, 63, 56], [254, 60, 58], [253, 58, 61], [253, 55, 64], [252, 53, 66], [252, 50, 69], [251, 48, 72], [250, 46, 74], [249, 43, 77], [248, 41, 80], [247, 39, 83], [246, 37, 86], [245, 35, 89], [244, 33, 92], [243, 31, 94], [241, 29, 97], [240, 27, 100], [238, 25, 103], [237, 23, 106], [235, 22, 109], [234, 20, 112], [232, 18, 115], [230, 17, 118], [228, 15, 122], [226, 14, 125], [224, 13, 128], [222, 11, 131], [220, 10, 134], [218, 9, 137], [216, 8, 140], [214, 7, 143], [212, 6, 146], [209, 5, 149], [207, 5, 152], [204, 4, 155], [202, 3, 158], [199, 3, 161], [197, 2, 164], [194, 2, 167], [192, 2, 169], [189, 1, 172], [186, 1, 175], [184, 1, 178], [181, 1, 181], [178, 1, 184], [175, 1, 186], [172, 1, 189], [170, 2, 192], [167, 2, 194], [164, 2, 197], [161, 3, 199], [158, 3, 202], [155, 4, 204], [152, 5, 207], [149, 5, 209], [146, 6, 211], [143, 7, 214], [140, 8, 216], [137, 9, 218], [134, 10, 220], [131, 11, 222], [128, 13, 224], [125, 14, 226], [122, 15, 228], [119, 17, 230], [116, 18, 232], [112, 20, 234], [109, 21, 235], [106, 23, 237], [103, 25, 238], [100, 27, 240], [98, 29, 241], [95, 31, 243], [92, 32, 244], [89, 35, 245], [86, 37, 246], [83, 39, 247], [80, 41, 248], [77, 43, 249], [75, 45, 250], [72, 48, 251], [69, 50, 252], [66, 53, 252], [64, 55, 253], [61, 58, 253], [59, 60, 254], [56, 63, 254], [54, 65, 255], [51, 68, 255], [49, 71, 255], [46, 74, 255], [44, 76, 255], [42, 79, 255], [40, 82, 255], [37, 85, 255], [35, 88, 254], [33, 91, 254], [31, 93, 253], [29, 96, 253], [27, 99, 252], [26, 102, 252], [24, 105, 251], [22, 108, 250]]

CLASSES = ['unlabeled', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'banner', 'blanket', 'branch', 'bridge', 'building-other', 'bush', 'cabinet', 'cage', 'cardboard', 'carpet', 'ceiling-other', 'ceiling-tile', 'cloth', 'clothes', 'clouds', 'counter', 'cupboard', 'curtain', 'desk-stuff', 'dirt', 'door-stuff', 'fence', 'floor-marble', 'floor-other', 'floor-stone', 'floor-tile', 'floor-wood', 'flower', 'fog', 'food-other', 'fruit', 'furniture-other', 'grass', 'gravel', 'ground-other', 'hill', 'house', 'leaves', 'light', 'mat', 'metal', 'mirror-stuff', 'moss', 'mountain', 'mud', 'napkin', 'net', 'paper', 'pavement', 'pillow', 'plant-other', 'plastic', 'platform', 'playingfield', 'railing', 'railroad', 'river', 'road', 'rock', 'roof', 'rug', 'salad', 'sand', 'sea', 'shelf', 'sky-other', 'skyscraper', 'snow', 'solid-other', 'stairs', 'stone', 'straw', 'structural-other', 'table', 'tent', 'textile-other', 'towel', 'tree', 'vegetable', 'wall-brick', 'wall-concrete', 'wall-other', 'wall-panel', 'wall-stone', 'wall-tile', 'wall-wood', 'water-other', 'waterdrops', 'window-blind', 'window-other', 'wood', 'other']

In [24]:
# with tqdm(unique_images, unit="Images") as pbar:
#     for idx, img_id in enumerate(pbar):
#         img = coco.loadImgs(img_id)[0]
#         filename = img['file_name']
#         coco_annIds = coco.getAnnIds(img_id, catIds=catIDs, iscrowd=None)
#         GT_annIds = cocoGt.getAnnIds(img_id, catIds=cat_stuffIDs, iscrowd=None)
#         coco_anns = coco.loadAnns(coco_annIds)
#         GT_anns = cocoGt.loadAnns(GT_annIds)
#         mask_A = np.zeros((img['height'], img['width']))
#         mask_B = np.zeros((img['height'], img['width']))
#         n, m = mask_B.shape
#         r = np.zeros_like(mask_A).astype(np.uint8)
#         g = np.zeros_like(mask_A).astype(np.uint8)
#         b = np.zeros_like(mask_A).astype(np.uint8)
#         for i in range(len(coco_anns)):
#             class_name = getClassName(coco_anns[i]['category_id'], cats)
#             pixel_value = filter_classes_map[class_name]
#             mask_A = np.maximum(coco.annToMask(coco_anns[i]) * pixel_value, mask_A)
#         for i in range(n):
#             for j in range(m):
#                 pixel = mask_A[i][j]
#                 if pixel != 0:
#                     id = filter_classes[pixel]
#                     color = color_mapping[filter_classes_map[id]]
#                     r[i][j] = color[0]
#                     g[i][j] = color[1]
#                     b[i][j] = color[2]
#
#         for i in range(len(GT_anns)):
#             class_name = getClassName(GT_anns[i]['category_id'], cats_stuff)
#             pixel_value = filter_classes_map[class_name]
#             mask_B = np.maximum(cocoGt.annToMask(GT_anns[i]) * pixel_value, mask_B)
#         for i in range(n):
#             for j in range(m):
#                 if mask_A[i][j] == 0:
#                     pixel = mask_B[i][j]
#                     if pixel != 0:
#                         id = filter_classes[pixel]
#                         color = color_mapping[filter_classes_map[id]]
#                         r[i][j] = color[0]
#                         g[i][j] = color[1]
#                         b[i][j] = color[2]
#
#
#         new_mask = np.stack([r,g,b],axis=2)
#
#         dir = 'C:\\Users\\SamuelMcmurray\\Documents\\SchoolStuff\\JU\\DeepLearning\\Project\\deeplearning_project\\coco\\coco2017\\masks\\val2017\\'
#         file = dir + filename
#         cv2.imwrite(file, new_mask)

In [25]:
globals()['IMAGE_SIZE'] = 128
globals()['BATCH_SIZE'] = 6
globals()['NUM_CLASSES'] = 173
globals()['IMAGE_DIR'] = r"C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017"
globals()['MASK_DIR'] = r"C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\masks"
globals()['NUM_TRAIN_IMAGES'] = 75000
globals()['NUM_VAL_IMAGES'] = 4999
hist_writer = tf.summary.create_file_writer("logs\\hist\\")
scalar_writer = tf.summary.create_file_writer("logs\\scalar\\")

In [26]:
# for i in range(5000):
#     # train_images = sorted(glob(os.path.join(IMAGE_DIR, "train2017/*")))[:NUM_TRAIN_IMAGES]
#     # train_masks = sorted(glob(os.path.join(MASK_DIR, "train2017/*")))[:NUM_TRAIN_IMAGES]
#     val_images = sorted(glob(os.path.join(IMAGE_DIR, "val2017/*")))[: NUM_VAL_IMAGES]
#     val_masks = sorted(glob(os.path.join(MASK_DIR, "val2017/*")))[:NUM_VAL_IMAGES]
#     print(i)
#     count = 0
#     for image, mask in zip(val_images, val_masks):
#         count += 1
#         image_end = image.split("\\")
#         mask_end = mask.split("\\")
#         if image_end[-1] != mask_end[-1]:
#             print(image_end[-1], mask_end[-1])
#             destination = r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\othertrain2017\val2017'
#             shutil.move(image, destination)
#             break

In [27]:
train_images = sorted(glob(os.path.join(IMAGE_DIR, "train2017/*")))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob(os.path.join(MASK_DIR, "train2017/*")))[:NUM_TRAIN_IMAGES]
val_images = sorted(glob(os.path.join(IMAGE_DIR, "val2017/*")))[: NUM_VAL_IMAGES]
val_masks = sorted(glob(os.path.join(MASK_DIR, "val2017/*")))[:NUM_VAL_IMAGES]

In [28]:
# COLOR_MAP = []
# CLASSES = []
# for cl in filter_classes_map:
#     CLASSES.append(cl)
#     COLOR_MAP.append([color_mapping[filter_classes_map[cl]][0], color_mapping[filter_classes_map[cl]][1], color_mapping[filter_classes_map[cl]][2]])
#     # print(color_mapping[filter_classes_map[cl]][0], color_mapping[filter_classes_map[cl]][1], color_mapping[filter_classes_map[cl]][2])
#
# print(CLASSES)
# print(len(CLASSES))
# print(COLOR_MAP)
# print(len(COLOR_MAP))

In [29]:
def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    x = x.astype(np.float32)

    return x

In [30]:
def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE), interpolation=3)

    # path = r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\othertrain2017'
    # save_image_path = os.path.join(path, "original.jpg")
    # cv2.imwrite(save_image_path, x)

    output = []
    for i, color in enumerate(COLOR_MAP):
        c_map = np.all(np.equal(x,color), axis=-1)
        # if True in c_map:
        #     print(CLASSES[i])
        output.append(c_map)
        # save_image_path = os.path.join(path, f"{i}.png")
        # cv2.imwrite(save_image_path, c_map * 255)

    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)

    return output

In [31]:
def preprocess(x,y):
    def f(x,y):
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_mask(y)

        return x, y

    image, mask = tf.numpy_function(f, [x,y], [tf.float32, tf.uint8])
    image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    mask.set_shape([IMAGE_SIZE, IMAGE_SIZE, NUM_CLASSES])
    return image, mask

In [32]:
def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.prefetch(2)
    return dataset

In [33]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

In [34]:
print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <PrefetchDataset element_spec=(TensorSpec(shape=(6, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(6, 128, 128, 173), dtype=tf.uint8, name=None))>
Val Dataset: <PrefetchDataset element_spec=(TensorSpec(shape=(6, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(6, 128, 128, 173), dtype=tf.uint8, name=None))>


In [35]:
def build_adjacency_matrix(connection_dict, directed=True):
    # Connection is from  row TO collumn.
    all_words = np.unique(list(connection_dict.keys())) # Assumption! See text above
    M = len(all_words)


    word2idx = {word:i for (i,word) in enumerate(all_words)}
    idx2word = {i:word for (i,word) in enumerate(all_words)}
    adj_m = np.zeros((M,M))
    for leaf in connection_dict.keys():
        parent = connection_dict[leaf]

        adj_m[word2idx[parent],word2idx[leaf]] = 1 # Adds connection from parent to child
        if not directed:
            adj_m[word2idx[leaf],word2idx[parent]] = 1 # Adds connection back from child to parent
    return adj_m, word2idx, idx2word, all_words

In [36]:
def ASPP(inputs):
    shape = inputs.shape
    y_pool = AveragePooling2D(pool_size=(shape[1], shape[2]))(inputs)
    y_pool = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(y_pool)
    y_pool = BatchNormalization()(y_pool)
    y_pool = Activation('relu')(y_pool)
    y_pool = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y_pool)

    y_6 = Conv2D(filters=256, kernel_size=1, dilation_rate=6, padding='same', use_bias=False)(inputs)
    y_6 = BatchNormalization()(y_6)
    y_6 = Activation('relu')(y_6)

    y_12 = Conv2D(filters=256, kernel_size=1, dilation_rate=12, padding='same', use_bias=False)(inputs)
    y_12 = BatchNormalization()(y_12)
    y_12 = Activation('relu')(y_12)

    y_18 = Conv2D(filters=256, kernel_size=1, dilation_rate=18, padding='same', use_bias=False)(inputs)
    y_18 = BatchNormalization()(y_18)
    y_18 = Activation('relu')(y_18)

    y_24 = Conv2D(filters=256, kernel_size=1, dilation_rate=24, padding='same', use_bias=False)(inputs)
    y_24 = BatchNormalization()(y_24)
    y_24 = Activation('relu')(y_24)

    y = Concatenate()([y_pool, y_6, y_12, y_18, y_24])

    y = Conv2D(filters=256, kernel_size=1, dilation_rate=1, padding='same', use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    return y

In [37]:
def histogram(name, scalar):
    # TF summary histogram wrapper to see how the various layers in the SGR progress.

    def clip_fn(t,min_val, max_val):
        return tf.clip_by_value(t, min_val, max_val)
    with hist_writer.as_default():
        clip_dict = {
            'input_features':(0,10),
            'output_features':(-2,2),
            'applied_mapping':(-50,50),
            'compatibility':(-10,60),
            'evolved_feat':(-10,10),
            'encoded_concat_feat':(-150,150),
            'visual_feat':(0,250)
        }

        if 'feat' in name:
            tf.summary.histogram(name+'_clip50',clip_fn(scalar,-50,50), step=train_step)
            tf.summary.histogram(name+'_clip10',clip_fn(scalar,-10,10), step=train_step)
        if 'softmax' in name:
            tf.summary.histogram(name+'_clip.001', clip_fn(scalar,0,0.001), step=train_step)
            tf.summary.histogram(name+'_clip.01', clip_fn(scalar,0,0.01), step=train_step)
            tf.summary.histogram(name+'_clip.4', clip_fn(scalar,0,0.4), step=train_step)

        else:#name in zooms:
            tf.summary.histogram(name+'_clip1', clip_fn(scalar,-2,2), step=train_step)
            tf.summary.histogram(name+'_clip1', clip_fn(scalar,-1,1), step=train_step)
            tf.summary.histogram(name+'_clip5', clip_fn(scalar,-5,5), step=train_step)

    return tf.summary.histogram(name, scalar)

In [38]:
@tf.function
def plot_to_image(fig):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)

    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)

    return image

In [39]:
@tf.function
def image_grid(display_list):
    assert data.ndim == 4 # [batch, H, W, C]
    figure= plt.figure(figsize=(10,10))

    num_images = data.shape[0] * len(display_list)
    size = int(np.ceil(np.sqrt(num_images)))
    if len(display_list) == 3:
        title = "Image, Ground Truth, and Prediction"
    else:
        title = "Image and Ground Truth"

    for i in range(display_list[0].shape[0]):
        plt.subplot(size, size, i + len(display_list), title=title)
        for i in range(len(display_list)):
            _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=(10,10))
            plt.axis('off')
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            if display_list[i].shape[-1] == 3:
                axes[i].imshow(tf.keras.utils.array_to_img(display_list[i]))
            elif display_list[i].shape[-1] == 1:
                axes[i].imshow(display_list[i])
    return figure

In [40]:
@tf.function
def get_confusion_matrix(y, y_pred, color_map, feature_classes):
    y_pred_pos = keras.backend.clip(y_pred, 0, 1)
    y_pred_neg = 1 - y_pred_pos
    y_pos = keras.backend.clip(y, 0, 1)
    y_neg = 1 - y_pos
    tp = keras.backend.sum(y_pos * y_pred_pos)
    fp = keras.backend.sum(y_neg * y_pred_pos)
    fn = keras.backend.sum(y_pos * y_pred_neg)
    return (fp,fn,tp)

In [41]:
@tf.function
def plot_confusion_matrix(confm, feature_classes):
    size = len(feature_classes)
    figure = plt.figure(figsize=(size, size))
    plt.imshow(confm, interpolation="nearest", cmap=plt.cm.colors.BASE_COLORS)
    indices = keras.backend.arange(len(feature_classes))
    plt.xticks(indices, feature_classes, rotation=45)
    plt.yticks(indices, feature_classes)

    confm = np.around(confm / keras.backend.sum(confm, axis=1)[:, np.newaxis], decimals=3)

    threshold = confm.max() / 2.0
    for i in range(size):
        for j in range(size):
            color = "white" if confm[i,j] > threshold else "black"
            plt.text(i,j,confm[i,j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.xlabel("True Label")
    plt.ylabel("Predicted Label")
    conf_image = plot_to_image(figure)
    return conf_image

In [42]:
def SGRModel(input_shape):
    inputs = Input(input_shape)
    base_model = ResNet101(weights='resnet101_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_tensor=inputs)
    image_features = base_model.get_layer('conv5_block3_out').output

    x_a = ASPP(image_features)
    x_b = UpSampling2D((4,4), interpolation='bilinear')(x_a)
    x_b = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x_b = UpSampling2D((4,4), interpolation='bilinear')(x_b)
    x_b = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x_b, inspect_voting = VotingToSemantic()(x_a)
    x_b, inspect_reasoning = GraphReasoning()(x_b)
    x_b, inspect_local = SemanticToLocal()(x_a, x_b)

    x_b = UpSampling2D((8,8), interpolation='bilinear')(x_b)
    x_b = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x_b = UpSampling2D((4,4), interpolation='bilinear')(x_b)
    x_b = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    outputs = Conv2D(NUM_CLASSES, (1,1), activation="softmax")(x_b)

    return Model(inputs=inputs, outputs=outputs)

In [43]:
class VotingToSemantic(tf.Module):
    def __init__(self, name="voting"):
        super().__init__(name)
        self.M = 204
        self.Dc = 256
        self.to_inspect = {}
        with self.name_scope:
            self.conv1 = Conv2D(filters=self.M, kernel_size=1, padding='same', use_bias=False, name="voting_convolution")
            self.conv2 = Conv2D(filters=self.Dc, kernel_size=1, padding='same', use_bias=False, name="feature_convolution")
            self.softmax = Softmax(axis=-1, name="voting_softmax_target")
            self.relu = ReLU(name='visual_feat')
    @tf.Module.with_name_scope
    def __call__(self, x, *args, **kwargs):
        with tf.name_scope('handle_voting'):
            inputs_shape = x.shape

            # will have shape [?,H,W,M]
            votes = self.conv1(x)
            votes = self.softmax(votes)
            self.to_inspect['softmax_vote_activations'] = votes
            # shape [?,H,W,M]
            # histogram('vote_softmax', votes)

            votes = tf.reshape(votes, [tf.shape(x)[0], inputs_shape[1] * inputs_shape[2], self.M], name='voting_reshape')
            # shape [?,H*W,M]
            votes = tf.transpose(votes, [0, 2, 1])  # shape [?,M, H*W] # every pixel Xi distributes
            # voting over the M nodes. np.sum(0,:,0]) must be 1

        with tf.name_scope('assign_votes'):
            in_feat = self.conv2(x)
            # shape [H*W,Dc]
            in_feat = tf.reshape(in_feat, [tf.shape(x)[0], inputs_shape[1] * inputs_shape[2], self.Dc], name='assign_votes_reshape') # shape [?,H*W,Dc]
            in_feat = tf.matmul(votes, in_feat, name='applied_votes')  # shape [?,M,Dc]
            visual_features = self.relu(in_feat)  # shape [?,M,Dc]
            # histogram('visual_features', visual_features)
            self.to_inspect['visual_features'] = visual_features

        return visual_features, self.to_inspect


In [44]:
class GraphReasoning(tf.Module):
    def __init__(self, name="voting"):
        super().__init__(name)
        self.M = 204
        self.Dc = 256
        self.K = 100
        self.to_inspect = {}
        with self.name_scope:
            self.conv1 = Conv1D(filters=self.Dc, kernel_size=1, activation='linear', name="graph_convolution")
            self.concat = Concatenate(axis=-1, name='concat_embed')
            self.fasttext = Embedding(self.M + 1, self.K, name="embedding")
            self.softmax = Softmax(axis=-1, name="voting_softmax_target")
            self.relu = ReLU(name='evolved_feat')

    def norm_adjacency(self, adjacency_matrix):
        a = adjacency_matrix  # MxM
        assert a.shape[0] == a.shape[1], 'adjacency matrix should be symmetirc'
        a += np.identity(a.shape[0])  # add identity connections
        tf_adj = tf.constant(a, dtype=tf.float32)
        norm_adj = self.row_normalize(tf_adj)
        return norm_adj


    def row_normalize(self, mat):
        Q = tf.reduce_sum(mat, axis=-1)
        Q = tf.cast(Q, tf.float32)
        sQ = tf.math.rsqrt(Q)
        sQ = linalg.diag(sQ)  # make diagonal mat
        return linalg.matmul(sQ, linalg.matmul(mat, sQ, name='right_side_norm'), name='left_side_norm')


    @tf.Module.with_name_scope
    def __call__(self, x, *args, **kwargs):
        with tf.name_scope('reasoning'):
            inputs_shape = x.shape
            adjacency_matrix, word_to_index, index_to_word, all_words = build_adjacency_matrix(all_dict)

            vocab_size = self.M + 2 # 2 extra variables
            vectorize_layer = TextVectorization(
                max_tokens=vocab_size,
                output_mode='int',
                vocabulary=all_words,
                output_sequence_length=1,
            )
            vectorize_layer.trainable = False
            vectorized_words = vectorize_layer(all_words)
            vectorized_words = tf.transpose(vectorized_words)

            fasttext_embeddings = self.fasttext(vectorized_words)
            fasttext_embeddings = tf.tile(fasttext_embeddings, [tf.shape(x)[0], 1, 1])

            concat_feat = self.concat([fasttext_embeddings, x])  # shape [b,M,Dc+K]
            # with hist_writer.as_default():
            #     tf.summary.histogram('concat_features', concat_feat, step=train_step)
            self.to_inspect['embed_concat_feat'] = concat_feat

            with tf.name_scope('encode_concat'):
                # shape [b*M,Dc+K]
                concat_feat = tf.reshape(concat_feat, [tf.shape(x)[0], self.M, self.Dc + self.K], name="encode_concat_reshape")
                Wg = self.conv1(concat_feat)
                # with hist_writer.as_default():
                #     tf.summary.histogram('Wg_encode_concat', Wg, step=train_step)

            # histogram('encoded_concat_feat', Wg)

            self.to_inspect['embed_transformed_feat'] = Wg

            # norm_adj has shape MxM
            with tf.name_scope('graph_convolution'):
                norm_adj = self.norm_adjacency(adjacency_matrix)
                norm_adj = tf.expand_dims(norm_adj, axis=0)

                expanded_norm_adj = tf.tile(norm_adj,[tf.shape(x)[0],1,1]) # shape [b,M,M] @ [b,M,Dc] --> [b,M,Dc] # approach two

                evolved_feat = tf.linalg.matmul(expanded_norm_adj, Wg,name='matmul_adj_feat') # [b,M,Dc]
                # histogram('evolved_feat',evolved_feat)

            self.to_inspect['embed_evolved_feat'] = evolved_feat

            # histogram('evolved_feat_before_relu', evolved_feat)
            evolved_feat = self.relu(evolved_feat)  # shape [b,M,Dc]

            # histogram('evolved_feat', evolved_feat)


        return evolved_feat, self.to_inspect

In [45]:
class SemanticToLocal(tf.Module):
    def __init__(self, name="voting"):
        super().__init__(name)
        self.M = 204
        self.Dc = 256
        self.Dl = 256
        self.to_inspect = {}
        with self.name_scope:
            self.conv1 = Conv1D(filters=self.Dl, kernel_size=1, padding='same', use_bias=False, name="Wsp")
            self.conv2 = Conv2D(filters=1, kernel_size=(1,1), padding='same', use_bias=False, name="Ws")
            self.concat = Concatenate(axis=-1,name='concat_inp')
            self.softmax = Softmax(axis=-1, name="voting_softmax_target")
            self.relu = ReLU(name='applied_mapping')

    def compute_compat_batches(self, input_feats ,evolved_feats):
        if input_feats.shape[0] is None:
            x = input_feats

            input_feats = tf.reshape(input_feats, [tf.shape(x)[0],tf.shape(x)[1] * tf.shape(x)[2], self.Dl], name="mapping_calc_reshape") # [H*W,Dl]
            input_feats = tf.tile(tf.expand_dims(input_feats,2),[1,1,self.M,1]) # [H*W,M,Dl]

            evolved_feats = tf.expand_dims(evolved_feats, 1)
            evolved_feats = tf.tile(evolved_feats,[1, tf.shape(x)[1]*tf.shape(x)[2],1,1]) # [H*W,M,Dc]

            concatenated = self.concat([evolved_feats, input_feats])
            concatenated = self.conv2(concatenated)
            return concatenated

        input_feats_stack = []
        for batch in input_feats:
            sh = tf.shape(batch)
            batch = tf.reshape(batch,[sh[0]*sh[1],sh[2]]) # shape [H*W,Dl]
            batch = tf.expand_dims(batch,0)
            batch = tf.tile(batch,[1,self.M,1]) # has shape [H*W,M,Dl]

            input_feats_stack.append(batch)
        input_feats = tf.stack(input_feats_stack, axis=0)

        evolved_feats_stack = []
        for batch in evolved_feats:
            sh = tf.shape(batch)
            batch = tf.expand_dims(batch,0)
            batch = tf.tile(batch,[sh[0]*sh[1],1,1]) # sh [H*W,M,Dc]
            evolved_feats_stack.append(batch)
        evolved_feats = tf.stack(evolved_feats_stack, axis=0)

        concatenated = self.concat([evolved_feats, input_feats])# (H*WxMx(Dc+Dl))
        concatenated = self.conv2(concatenated) # has shape (H*W,M,1)

        return concatenated

    @tf.Module.with_name_scope
    def __call__(self, x_a, x_b, *args, **kwargs):
        inputs_shape_a = x_a.shape
        inputs_shape_b = x_a.shape
        with tf.name_scope('calc_mapping'):
            concatenated = self.compute_compat_batches(x_a, x_b)
            # with scalar_writer.as_default():
            #     tf.summary.scalar('compat/sparsity', tf.nn.zero_fraction(concatenated))

            # histogram('compatibility',concatenated)
            self.to_inspect['compatibility'] = concatenated

            mapping = Softmax(axis=1, name='mapping_softmax')(concatenated)# (b,H*W, M, M) ! note axis, we want to take softmax w.r.t. other pixels,
            # with scalar_writer.as_default():
            #     tf.summary.scalar('mapping/sparsity', tf.nn.zero_fraction(mapping), step=train_step)

            self.to_inspect['mapping_target'] = mapping
            # histogram('mapping_softmax', mapping)

            with tf.name_scope('row_normalize_mapping'):
                #mapping has shape [b,H*W,M,M]
                mapping = tf.norm(mapping,ord=1,axis=-1) # has shape [b,H*W,M] holding sums along M dim
                mapping = tf.nn.l2_normalize(mapping,axis=2) # does l2 normalization along the rows.
                # histogram('mapping_softmax_row_norm', mapping)
            self.to_inspect['mapping_row_norm'] = mapping  # shape [b,H,W,M]


        with tf.name_scope('apply_mapping'):
            Wsp = self.conv1(x_b)

            # with hist_writer.as_default():
            #     tf.summary.histogram('W_transf_evolved', Wsp, step=train_step)
            self.to_inspect['transf_evolved'] = Wsp

            applied_mapping = tf.matmul(mapping, Wsp, name='applied_mapping')  # bx(H*W)xDl
            self.to_inspect['applied_mapping_before_relu'] = applied_mapping
            # histogram('applied_mapping_before_relu', applied_mapping)
            applied_mapping = self.relu(applied_mapping)  # bx(H*W)xDl
            applied_mapping = tf.reshape(applied_mapping, [tf.shape(x_a)[0], inputs_shape_a[1], inputs_shape_a[2], self.Dl],
                                         name='mapped_feat_target')  # b,HxWxDl    # Look at feature maps! (there's a lot so maybe difficult)

            # histogram('SGR_output_features', applied_mapping)
            self.to_inspect['applied_mapping'] = applied_mapping
            self.to_inspect['SGR_input'] = x_a
            # with scalar_writer.as_default():
            #     tf.summary.scalar('applied_mapping/sparsity', tf.nn.zero_fraction(applied_mapping), step=train_step)
        return (x_a + applied_mapping), self.to_inspect

In [46]:
# image = Image.open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\masks\train2017\000000000009.jpg')
# img = np.asarray(image)
# plt.axis('off')
# cmap = plt.get_cmap("jet")
# rgba_img = cmap(img)
# rgb_img = np.delete(rgba_img, 3,2)
# plt.imshow(rgb_img)

In [47]:
# image = Image.open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\masks\train2017\000000000030.jpg')
# img = np.asarray(image)
# semantic_img = []
# for i in range(len(img[0])):
#     for j, cl in enumerate(classes_map):
#         if filter_classes_map[cl] == img[0][i]:
#             semantic_img.append(cl)
# print(semantic_img)
# image = Image.open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\masks\train2017\000000000030.jpg')
# img = np.asarray(image)
# plt.axis('off')
# cmap = plt.get_cmap("jet")
# rgba_img = cmap(img)
# rgb_img = np.delete(rgba_img, 3,2)
# plt.imshow(rgb_img)

In [48]:
# image = Image.open(r'C:\Users\SamuelMcmurray\Documents\SchoolStuff\JU\DeepLearning\Project\deeplearning_project\coco\coco2017\train2017\000000000030.jpg')
# plt.axis('off')
# plt.imshow(image)

In [49]:
model = SGRModel((IMAGE_SIZE,IMAGE_SIZE,3))
num_epochs = 64
optimizer = SGD(learning_rate=2.5e-3)
loss_fn = keras.losses.CategoricalCrossentropy()
acc_metric = keras.metrics.CategoricalCrossentropy()
meanIoU = keras.metrics.MeanIoU(num_classes=NUM_CLASSES, name="MeanIoU")
# train_writer = tf.summary.create_file_writer("logs/train/")
# test_writer = tf.summary.create_file_writer("logs/test/")
# chkp = tf.train.Checkpoint(step = tf.Variable(1), optimizer= optimizer, model=model)
# manager = tf.train.CheckpointManager(chkp, 'tf_chkps', max_to_keep=3)
# image_writer = tf.summary.create_file_writer("logs\\images\\")
# graph_writer = tf.summary.create_file_writer("logs\\graph_vis")

In [50]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']         

In [51]:
keras.utils.plot_model(model, show_shapes=True)

In [52]:
# def dice_coef(y_true, y_pred):
#     y_true_f = keras.backend.flatten(y_true)
#     y_pred_f = keras.backend.flatten(y_pred)
#     intersection = keras.backend.sum(y_true_f* y_pred_f)
#     val = (2. * intersection + 0.00001 / (keras.backend.sum(y_true_f * y_true_f) + keras.backend.sum(y_pred_f * y_pred_f) + 0.00001))
#     return val

In [53]:
# def jacard_similarity(y_true, y_pred):
#     """
#      Intersection-Over-Union (IoU), also known as the Jaccard Index
#     """
#     y_true_f = keras.backend.flatten(y_true)
#     y_pred_f = keras.backend.flatten(y_pred)
#
#     intersection = keras.backend.sum(y_true_f * y_pred_f)
#     union = keras.backend.sum((y_true_f + y_pred_f) - (y_true_f * y_pred_f))
#     return intersection / union
#
# def jacard_loss(y_true, y_pred):
#     """
#      Intersection-Over-Union (IoU), also known as the Jaccard loss
#     """
#     return 1 - jacard_similarity(y_true, y_pred)

In [54]:
# @tf.function
# def train_step(model, x, y, optimizer):
#     with tf.GradientTape() as tape:
#         y_pred = model(x, training=True)
#         loss = loss_fn(y, y_pred)
#         accuracy = acc_metric(y, y_pred)
#         # jacard = jacard_similarity(y, y_pred)
#         # dice = dice_coef(y, y_pred)
#         mean_iou = meanIoU(tf.argmax(y, axis=-1), tf.argmax(y_pred, axis=-1))
#         # confusion += get_confusion_matrix(y, y_pred, color_mapping, filter_classes_map)
#
#     variables = model.trainable_weights
#     gradients = tape.gradient(loss, variables)
#     optimizer.apply_gradients(zip(gradients, variables))
#     return loss, accuracy, mean_iou

In [55]:
# def train_and_checkpoints(model, manager):
#     chkp.restore(manager.latest_checkpoint)
#     if manager.latest_checkpoint:
#         print("Restored from {}".format(manager.latest_checkpoint))
#     else:
#         print("Initializing from scratch.")
#
#
#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch + 1, num_epochs))
#
#         with tqdm(train_dataset, unit="batches") as pbar:
#             losses = []
#             accuracies = []
#             # dices = []
#             # jacards = []
#             meanIoUs = []
#             # confusion = []
#             step = 0
#             for idx, (x, y) in enumerate(pbar):
#                 # figure = image_grid([x,y])
#                 # with image_writer.as_default():
#                 #     tf.summary.image(name="Image Visualization", data=plot_to_image(figure), step=step)
#                 loss, accuracy, mean_iou = train_step(model, x, y, optimizer)
#                 losses.append(keras.backend.eval(loss))
#                 accuracies.append(keras.backend.eval(accuracy))
#                 # jacards.append(keras.backend.eval(jacard))
#                 # dices.append(keras.backend.eval(dice))
#                 meanIoUs.append(keras.backend.eval(mean_iou))
#                 if int(step) % 10 == 0:
#                     save_path = manager.save()
#                     print("Saved checkpoint for step {}: {}".format(int(step), save_path))
#                     print("loss {:1.2f}".format(keras.backend.eval(loss)))
#                 step += 1
#                 pbar.set_postfix({
#                     'loss': keras.backend.eval(loss),
#                     'accuracy': keras.backend.eval(accuracy),
#                     # 'jacard': keras.backend.eval(jacard),
#                     # 'dice': keras.backend.eval(dice),
#                     'mean IoU': keras.backend.eval(mean_iou)})
#
#
#             with train_writer.as_default():
#                 tf.summary.scalar("Loss", np.mean(losses), step=epoch)
#                 tf.summary.scalar("Accuracy", np.mean(accuracies), step=epoch)
#                 tf.summary.scalar("MeanIoU", np.mean(meanIoUs), step=epoch)
#                 # tf.summary.scalar("Dice", np.mean(dices), step=epoch)
#                 # tf.summary.scalar("Jacard", np.mean(jacards), step=epoch)
#                 # tf.summary.image("Confusion Matrix", plot_confusion_matrix(confusion / idx, color_mapping, filter_classes_map), step=epoch)
#             for batch_idx, (x,y) in enumerate(val_dataset):
#                 y_pred = model(x, training=False)
#                 loss = loss_fn(y, y_pred)
#                 accuracy = acc_metric(y, y_pred)
#                 # jacard = jacard_similarity(y, y_pred)
#                 # dice = dice_coef(y, y_pred)
#                 mean_iou = meanIoU(tf.argmax(y, axis=-1), tf.argmax(y_pred, axis=-1)).numpy()
#
#             with test_writer.as_default():
#                 tf.summary.scalar("Val_Loss", loss, step=epoch)
#                 tf.summary.scalar("Val_Accuracy", accuracy, step=epoch)
#                 tf.summary.scalar("Val_MeanIoU", mean_iou, step=epoch)
#                 # tf.summary.scalar("Val_Dice", dice, step=epoch)
#                 # tf.summary.scalar("Val_Jacard", jacard, step=epoch)


In [56]:
# train_and_checkpoints(model, manager)

In [57]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="tb_callback_dir,", histogram_freq=1)
optimizer = SGD(learning_rate=2.5e-3)
callbacks = [
    keras.callbacks.ModelCheckpoint("coco-stuff\\save_at_{epoch}.keras"),
    tensorboard_callback
]
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy", keras.metrics.MeanIoU(num_classes=NUM_CLASSES, name="MeanIoU")],
)
history = model.fit(
    train_dataset,
    epochs=num_epochs,
    callbacks=callbacks,
    validation_data=val_dataset,
    verbose=2
)
model.save("../models/sgr_resnet101.hdf5")

Epoch 1/64
12500/12500 - 2883s - loss: 0.9083 - accuracy: 0.2135 - MeanIoU: 0.4991 - val_loss: 0.8993 - val_accuracy: 0.2146 - val_MeanIoU: 0.4991 - 2883s/epoch - 231ms/step
Epoch 2/64
12500/12500 - 2966s - loss: 0.9863 - accuracy: 0.1817 - MeanIoU: 0.4991 - val_loss: 1.0866 - val_accuracy: 0.0950 - val_MeanIoU: 0.4991 - 2966s/epoch - 237ms/step
Epoch 3/64
12500/12500 - 2971s - loss: 7.5581 - accuracy: 0.0871 - MeanIoU: 0.4995 - val_loss: 41.6976 - val_accuracy: 0.0144 - val_MeanIoU: 0.4991 - 2971s/epoch - 238ms/step
Epoch 4/64
12500/12500 - 2962s - loss: 337.2415 - accuracy: 0.0356 - MeanIoU: 0.5045 - val_loss: 1249.7153 - val_accuracy: 0.0659 - val_MeanIoU: 0.5232 - 2962s/epoch - 237ms/step
Epoch 5/64
12500/12500 - 3056s - loss: 11440.3223 - accuracy: 0.0283 - MeanIoU: 0.5004 - val_loss: 35115.6211 - val_accuracy: 1.5045e-05 - val_MeanIoU: 0.4962 - 3056s/epoch - 244ms/step
Epoch 6/64


KeyboardInterrupt: 

In [ ]:
model.save("../models/sgr_resnet101.hdf5")

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
mean_iou = history.history["MeanIoU"]
val_mean_iou = history.history["val_MeanIoU"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'g', label='Validation Accuracy')
plt.plot(epochs, mean_iou, 'y', label='MeanIoU')
plt.plot(epochs, mean_iou, 'r', label='Validation MeanIoU')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(history.history["MeanIoU"])
plt.title("MeanIoU")
plt.ylabel("MeanIoU")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(history.history["val_MeanIoU"])
plt.title("Validation MeanIoU")
plt.ylabel("val_MeanIoU")
plt.xlabel("epoch")
plt.show()

In [ ]:
# def grayscale_to_rgb(pred,classes, colormap):
#     h,w, _ = pred.shape
#     pred = pred.astype(np.int32)
#
#     output=[]
#     for i, pixel in enumerate(pred.flatten()):
#         output.append(colormap[pixel])
#
#     output = np.reshape(output, (h,w,3))
#     return output

In [ ]:
# def save_image_pred(image_x, mask_x, pred, save_image_path):
#     h,w,_ = image_x.shape
#     line = np.ones((h,w,3)) * 255
#     pred = np.expand_dims(pred, axis=-1)
#     pred = grayscale_to_rgb(pred,CLASSES, COLOR_MAP)
#     print(pred.shape)
#     cat_images = np.concatenate([image_x, line, mask_x, line, pred])
#     cv2.imwrite(save_image_path, cat_images)


In [ ]:
# SCORE = []
# for x, y in tqdm(zip(val_images, val_masks), total=1000):
#     name = x.split("/")[-1].split(".")[0]
#     image = cv2.imread(x, cv2.IMREAD_COLOR)
#     image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
#     image_x = image
#     image = image/255.0
#     image = np.expand_dims(image, axis=0)
#
#     mask = cv2.imread(y, cv2.IMREAD_COLOR)
#     mask = cv2.imread(y, cv2.IMREAD_COLOR)
#     mask = cv2.resize(mask, (IMAGE_SIZE, IMAGE_SIZE))
#     mask_x = mask
#     onehot_mask = []
#     for color in COLOR_MAP:
#         cmap = np.all(np.equal(mask, color), axis=-1)
#         onehot_mask.append(cmap)
#     onehot_mask = np.stack(onehot_mask, axis=-1)
#     onehot_mask = np.argmax(onehot_mask, axis=-1)
#     onehot_mask = onehot_mask.astype(np.int32)
#
#     pred = model.predict(image, verbose=0)[0]
#     pred = np.argmax(pred, axis=-1)
#     pred = pred.astype(np.float32)
#
#     save_image_path = os.path.join("result","sgr", "{}.png".format(name))
#     print(save_image_path)
#     save_image_pred(image_x, mask_x, pred, save_image_path)